#Load dataset (VoxCeleb1)

In [ ]:
!pip install datasets

In [35]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [36]:
import tarfile

file = tarfile.open('drive/MyDrive/Speech_understanding/VoxCeleb1_subset.tar-tron’s MacBook Pro.gz')

In [37]:
file.extractall('./VoxCeleb1')
file.close()

#Load Pre-trained Models

##ECAPA

In [38]:
# !pip install git+https://github.com/speechbrain/speechbrain.git@develop

In [52]:
from speechbrain.inference.speaker import SpeakerRecognition
verification = SpeakerRecognition.from_hparams(source="speechbrain/spkrec-ecapa-voxceleb", savedir="pretrained_models/spkrec-ecapa-voxceleb")

#Compute EER

In [58]:
import os
import torchaudio
import numpy as np

from tqdm import tqdm
from sklearn.metrics import roc_curve

In [47]:
def compute_eer(y_true, y_score):
    fpr, tpr, thresholds = roc_curve(y_true, y_score, pos_label=1)
    eer = fpr[np.nanargmin(np.abs(fpr - (1 - tpr)))]
    return eer

In [67]:
dataset_dir = "./VoxCeleb1/VoxCeleb1_subset"
test_dir = os.path.join(dataset_dir, "test")

with open(os.path.join(dataset_dir, 'list_test_hard.txt'), "r") as f:
    pairs = [line.strip().split() for line in f]

In [68]:
test_scores = []
test_labels = []
for pair in tqdm(pairs):
    file1_path = os.path.join(test_dir, pair[1])
    file2_path = os.path.join(test_dir, pair[2])
    if os.path.exists(file1_path) and os.path.exists(file2_path):
      score, _ = verification.verify_files(file1_path, file2_path)
      test_scores.append(score)
      test_labels.append(int(pair[0]))

100%|██████████| 10000/10000 [04:45<00:00, 34.99it/s]


In [69]:
test_eer = compute_eer(test_labels, test_scores)

In [71]:
test_eer/ 100

0.5